# MI
**_Ahmed - Luisa - Myria_**

## Exercise H62.1: Long short-term memory (LSTM)

### a) Creating training and validation datasets

In [19]:
import numpy as np

def generate_series(series_size, total_count):
    return [np.random.randint(0, 10, series_size) for i in range(total_count)]

def evaluate_series(series):
    return 1 if sum(series) >= 100 else 0

def generate_input_output(collection_of_series):
    output = [evaluate_series(series) for series in collection_of_series]
    return list(zip(collection_of_series, output))
        
training_dataset = generate_input_output(generate_series(30, 8000))
validation_dataset = generate_input_output(generate_series(30, 2000))

### b) Building an RNN